In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

# Spark session
spark = SparkSession.builder \
    .appName("GoldLayer") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Read from silver layer
silver_path = "/opt/bitnami/spark/data/silver"
df_silver = spark.readStream.format("delta").load(silver_path)

# Business aggregations
df_gold = df_silver.groupBy("crypto_base", "quote_currency") \
    .agg(
        avg("amount").alias("avg_price"),
        max("amount").alias("max_price"),
        min("amount").alias("min_price"),
        max("timestamp").alias("latest_timestamp")
    )

# Write to gold layer
gold_path = "/opt/bitnami/spark/data/gold"
checkpoint_path = "/opt/bitnami/spark/data/gold_checkpoint"

query = df_gold.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", checkpoint_path) \
    .start(gold_path)

query.awaitTermination()